# data upload

In [27]:
import os
base = os.path.join('C:\\', 'Users', 'steve', 'Documents', 'reliant-scrape')
os.chdir(base)

In [24]:
from sqlalchemy import create_engine, types
from datetime import datetime, timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import reliant_scrape as rs
import pyodbc
import pandas as pd
import numpy as np
import pprint
import yaml
import json
import sys
import os

base = os.path.join('C:\\', 'Users', 'steve', 'Documents', 'reliant-scrape')
data = os.path.join(base, 'data')
pp = pprint.PrettyPrinter(indent = 1)

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader = yaml.FullLoader)
        
with open('db_creds.json', 'r') as f:
    db_creds = json.load(f)
    
print('loaded credentials & imported modules.')

loaded credentials & imported modules.


In [26]:
mydata = rs.sql_query("""SELECT * FROM reliant_energy_db.daily_use""", db_creds)

AttributeError: module 'reliant_scrape' has no attribute 'sql_query'

In [6]:
os.chdir(data)
stage = pd.read_csv('master_usage_02212021.csv')
stage['Date'] = pd.to_datetime(stage['Date'])

In [22]:
result = rs.mysql_query("""SELECT MIN(Date) as min_date, MAX(Date) as max_date, COUNT(*) as count 
                    FROM reliant_energy_db.daily_use""", db_creds)

In [23]:
recent = [d > result.max_date[0] for d in stage.Date]
merge = stage.iloc[recent,:]

In [7]:
stage

,Date,Usage (kWh),Cost ($),Hi,Low
0,2020-08-25 00:00:00,1.1,0.13,94,79
1,2020-08-25 01:00:00,1.2,0.14,94,79
2,2020-08-25 02:00:00,1.0,0.12,94,79
3,2020-08-25 03:00:00,1.0,0.12,94,79
4,2020-08-25 04:00:00,1.5,0.18,94,79
...,...,...,...,...,...
4267,2021-02-19 19:00:00,0.5,0.06,50,27
4268,2021-02-19 20:00:00,0.5,0.06,50,27
4269,2021-02-19 21:00:00,0.5,0.07,50,27
4270,2021-02-19 22:00:00,0.5,0.06,50,27


In [10]:
rs.table_upload(stage, 'raspberrypi', 'reliant_energy', db_creds)
print('uploaded data.')

wrote df to sql table.
uploaded data.
